In [ ]:
# put this code in a .bat file to run the script when starting the computer. The if clause keeps the
# command line minimized. 'C:\Users\ufrecht\Anaconda3\python' is the location of python.exe and 
# 'C:\Users\ufrecht\Desktop\Dateien\arXiv_downloader.py' the location and name of the python script.

#if not DEFINED IS_MINIMIZED set IS_MINIMIZED=1 && start "" /min "%~dpnx0" %* && exit
#C:\Users\ufrecht\Anaconda3\python C:\Users\ufrecht\Desktop\Dateien\arXiv_downloader.py
#exit


from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from pathlib import Path
import requests
import os

def find_index(text, word, second_word):
    '''Takes the string "text" as input and searches for the first appearance of "word".
    Once found the program goes back to "second word" and returns idx, the beginning of the "second_word"
    and "end_index" the index of the ending of "second word". If the text does not contain both words, the function 
    returns (None, None)'''
     if (word in text) and (second_word in text):
        a = re.search(r'' + word +'', text)
        start_index = a.span()[0]
        end_index=a.span()[1]
        b=text[start_index::-1]
        a = re.search(r''+second_word[::-1]+'', b)
        idx=start_index-a.span()[1]+1
        return (idx, end_index)
     return (None, None)

def url2text(url):
    '''Download the text on internet page with "url" and return this text'''
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()

def download_pdf(url,filename):
    '''Dowload pdf file from "url" and save as "filename", which includes the path'''
    filename = Path(filename+'.pdf')
    response = requests.get(url)
    filename.write_bytes(response.content)

def search_all_words(url,word, text):
    '''Download text from "url" and search index of the first "word" and secon word arXiv. Go through text
    download pdfs for all instances of this word '''
    words_left=True
    while(words_left):
        second_word='arXiv'
        idx, end_index=find_index(text.lower(), word.lower(), 
        second_word.lower())
        try:
            arxiv_number=text[idx+6:idx+16]
            url='https://arxiv.org/pdf/'+arxiv_number+'.pdf'
            Desktop_path = os.path.expanduser("~/Desktop")
            file_path=Desktop_path+'/New_on_arXiv'
            if not os.path.exists(file_path):
                os.makedirs(file_path)
            download_pdf(url,file_path+'/'+'arXiv_'+arxiv_number)
            text=text[end_index::]
        except:
            words_left=False


urls=["https://arxiv.org/list/physics.atom-ph/new", "https://arxiv.org/list/quant-ph/new"]
words=['redshift', 'red-shift', 'red shift', 'free fall', 'free-fall', 'equivalence principle', 'variational', 'nisq', 'reinforcement','computing', 'machine-learning', 'machine learning']


for url in urls:
    text=url2text(url)
    for word in words:
        search_all_words(url,word, text)



print('>> done')


